#Продвинутый Python, ДЗ-3

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно рисовать графики с помощью matplotlib или же seaborn. Обратите внимание, что все графики и оси должны быть подписаны, графики должны выглядеть аккуратно (за это будем штрафовать)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [ ]:
!pip install wget

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')



# Первое Задание

delivered_orders = orders[orders['order_status'] == 'delivered']
merged_data = delivered_orders.merge(items, on='order_id')

product_purchase_count = merged_data['product_id'].value_counts().reset_index()
product_purchase_count.columns = ['product_id', 'purchase_count']

merged_products = product_purchase_count.merge(products, on='product_id')

fig, axs = plt.subplots(3, 1, figsize=(14, 18))
# по количеству фотографий
sns.barplot(x='product_photos_qty', y='purchase_count', data=merged_products, ax=axs[0], estimator=np.mean,
            errorbar=None)
axs[0].set_title('Зависимость между средним числом покупок товара и числом фотографий товара')
axs[0].set_xlabel('Число фотографий товара')
axs[0].set_ylabel('Среднее число покупок')

# по описанию
merged_products['description_length_grouped'] = (merged_products[
                                                     'product_description_lenght'] // 20) * 20  # группируем по 20
sns.barplot(x='description_length_grouped', y='purchase_count', data=merged_products, ax=axs[1], estimator=np.mean,
            errorbar=None)
axs[1].set_title('Зависимость между средним числом покупок товара и длинной описания')
axs[1].set_xlabel('Количество символов в описании(с шагом 20)')
axs[1].set_ylabel('Среднее число покупок')

# по имени
merged_products['name_length_grouped'] = (merged_products['product_name_lenght'] // 5) * 5  # группируем по 5
sns.barplot(x='name_length_grouped', y='purchase_count', data=merged_products, ax=axs[2], estimator=np.mean,
            errorbar=None)
axs[2].set_title('Зависимость между средним числом покупок товара и кол-вом символов в названии')
axs[2].set_xlabel('Количество символов в названии товара')
axs[2].set_ylabel('Среднее число покупок')

plt.tight_layout()
plt.show()

# 2 Задание
# Убираю not defined
f_payments = payments[
    (payments['payment_type'] != 'not_defined') &
    (payments['payment_value'] > 0)
    ]

m_payment = f_payments.groupby('payment_type')['payment_value'].mean()

plt.figure(figsize=(12, 6))
sns.barplot(x=m_payment.index, y=m_payment.values,
            hue=m_payment.index, palette="viridis", legend=False)
plt.title('Средняя сумма оплаты по типу оплаты (без "not defined" и нулевых значений)')
plt.xlabel('Тип оплаты')
plt.ylabel('Средняя сумма оплаты')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()

# 3 задание
order_payment_counts = payments.groupby('order_id').size().reset_index(name='num_payments')

# Больше одной оплаты
multiple_payments = order_payment_counts[order_payment_counts['num_payments'] > 1]

multiple_payments_details = payments[payments['order_id'].isin(multiple_payments['order_id'])]

total_payment_order = multiple_payments_details.groupby('order_id')['payment_value'].sum().reset_index()
# total_payment_order.head()

merged_payments = pd.merge(multiple_payments_details, total_payment_order, on='order_id', suffixes=('_individual',
                                                                                                    '_total'))

avg_payment_per_installment = payments.groupby('payment_installments')['payment_value'].mean().reset_index()

# График
plt.figure(figsize=(12, 6))
sns.barplot(data=avg_payment_per_installment, x='payment_installments', y='payment_value')
plt.title('Зависимость среднего чека от числа оплат')
plt.xlabel('Число оплат')
plt.ylabel('Средний чек')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()




ModuleNotFoundError: No module named 'wget'

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (20 баллов)

Визуализируйте зависимость между средним числом покупок товара и:

* числом фотографий товара

* кол-во символов в описании товара (аггрегируйте с шагом 20)

* кол-во символов в названии (аггрегируйте с шагом 5)

Для подсчета среднего числа покупок нужно посчитать по каждому товару число продаж, а затем взять среднее от него (учтите, что мы считаем только доставленные заказы)

## Задание 2 (10 баллов)

Отобразите среднюю сумму оплаты по типу оплаты (для этого вам нужна табличка payments)

## Задание 3 (25 баллов)

Обратите внимание на число платежей (payment_installments). Выясните, какая сумма оплаты указана: за целую покупку или только за ее часть? (Обратите внимание, что в данных есть определенная бага, которую нужно расследовать и учесть)

Также покажите средний чек в зависимости от числа оплат. Есть ли какая-то связь между ними?

## Задание 4 (15 баллов)

Визуализируйте зависимость между ценой заказа и ценой доставки по штатам покупки. Есть ли какая-нибудь зависимость и как вы можете ее обосновать?

## Задание 5 (15 баллов)

Отобразите распределение оценок для заказов, которые были доставлены вовремя и которые пришли с опозданием. Насколько влияет факт опоздания на оценку? (оценки берутся из таблицы reviews)

## Задание 6 (15 баллов)

Изобразите зависимость между ценой заказа и числом дней между покупкой и доставкой с разбивкой по признаку "есть заказ из другого штата". Есть ли зависимость?